In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.solutions.object_counter import ObjectCounter

model = YOLO(r"D:\multivariety_tracking\runs\segment\train20\weights\best.pt")

# Read video frames
video_path = r'D:\multivariety_tracking\segementation_project\test_1_30sec_tree_seg.mp4'
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"
opacity = 0.5
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
region_points = [(0, 0), (w, 0), (w, h), (0, h)]

n_video = cv2.VideoWriter(r"D:\multivariety_tracking\segementation_project\test_1_30sec_tree_seg_train_20.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
class_name_to_id = {0: 'banganapali', 1: 'javeri', 2: 'rumani'}
class_ids = list(class_name_to_id.keys())

object_counter = ObjectCounter(class_name_to_id, reg_pts=region_points,
                               count_reg_color=(255, 0, 255), 
                               count_txt_color=(0, 0, 0), 
                               count_bg_color=(255, 255, 255), 
                               line_thickness=None, 
                               track_thickness=2, 
                               view_img=True, 
                               view_in_counts=False, 
                               view_out_counts=False, 
                               draw_tracks=False, 
                               track_color=None, 
                               region_thickness=None, 
                               line_dist_thresh=15, 
                               cls_txtdisplay_gap=50)
colors = [[21, 142, 207], [64, 165, 62], [145, 141, 127], [255, 192, 203], [0, 255, 255], [128, 128, 128]]


frame_number = 0


while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    tracks = model.track(frame, persist=True, show=False, classes=class_ids)

    for result in tracks:
        if result.masks is not None and result.boxes is not None:
            for mask, box in zip(result.masks.xy, result.boxes):
             
              if mask.size != 0:  
                points = np.int32([mask.reshape(-1, 1, 2)])  
                color_number = class_ids.index(int(box.cls[0]))
                overlay = frame.copy()
                cv2.fillPoly(overlay, points, colors[color_number])

              
                frame = cv2.addWeighted(overlay, opacity, frame, 1 - opacity, 0)
    
  
    frame = object_counter.start_counting(frame, tracks)

    total_in_count = object_counter.in_counts
    total_out_count = object_counter.out_counts
    class_wise_counts = object_counter.class_wise_count

    font = cv2.FONT_HERSHEY_SIMPLEX
    initial_y_position = 50
    text_gap = 30  # Gap between each text line

    # Iterate through the counts dictionary and print class name and total count on the frame
    for i, (class_name, count_dict) in enumerate(class_wise_counts.items()):
        text = f"{class_name}: {count_dict['IN']+count_dict['OUT']}"
        position = (10, initial_y_position + i * text_gap)
        cv2.putText(frame, text, position, font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Write the processed frame to the output video
    n_video.write(frame)

    # Calculate and display progress
    frame_number += 1
    completion_percentage = (frame_number / total_frames) * 100
    remaining_frames = total_frames - frame_number
    print(f"Progress: {completion_percentage:.2f}% | Remaining frames: {remaining_frames}")

# Release resources
n_video.release()
cap.release()


WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'

Polygon Counter Initiated.

0: 384x640 2 javeris, 1 rumani, 88.6ms
Speed: 16.0ms preprocess, 88.6ms inference, 103.3ms postprocess per image at shape (1, 3, 384, 640)
Progress: 0.14% | Remaining frames: 719

0: 384x640 2 javeris, 1 rumani, 79.0ms
Speed: 6.0ms preprocess, 79.0ms inference, 13.0ms postprocess per image at shape (1, 3, 384, 640)
Progress: 0.28% | Remaining frames: 718

0: 384x640 2 javeris, 1 rumani, 82.0ms
Speed: 5.0ms preprocess, 82.0ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)
Progress: 0.42% | Remaining frames: 717

0: 384x640 2 jav